# CNN_Daily_Mail_Final_Model_1a_Sbert_Summarizer_Extractive_Summarization
    November 6, 2022 



This notebook has the following model built for an extractive summarizer based on Bert Sentence Transfomer Model from Hugging Face.

1. **SBertSummarizer** works by first embedding the sentences, then running a clustering algorithm, finding the sentences that are closest to the cluster's centroids. This library also uses coreference techniques, utilizing the https://github.com/huggingface/neuralcoref library to resolve words in summaries that need more context. 
2.**SentenceTransformers** is a Python framework for state-of-the-art sentence, text and image embeddings. The initial work is described in paper Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks. This framework can be used to compute sentence / text embeddings for more than 100 languages. **These** embeddings can then be compared e.g. with cosine-similarity to find sentences with a similar meaning. This can be useful for semantic textual similar, semantic search, or paraphrase mining.

Dataset Summary

The CNN / DailyMail Dataset is an English-language dataset containing just over 300k unique news articles as written by journalists at CNN and the Daily Mail. The current version supports both extractive and abstractive summarization, though the original version was created for machine reading and comprehension and abstractive question answering. 

Data Fields

    id: a string containing the heximal formated SHA1 hash of the url where the story was retrieved from
    article: a string containing the body of the news article
    highlights: a string containing the highlight of the article as written by the article author

Model Details

    Sentence Transformer: SBert Summarizer
    Pre-Training: all-MiniLM-L6-v2
    **Supervised**: Unsupervised Model
    Classification: Built-In Classifer for Sentences
    Trigram Blocking: Yes
    Fine Tuning: None
    Evaluation Metrics: RougeL and Cosine Similarity
    




# 1. Setup

#### This section install key libraries

In [ ]:
!pip install bert-extractive-summarizer --quiet

     |████████████████████████████████| 5.5 MB 5.9 MB/s 
     |████████████████████████████████| 7.6 MB 50.2 MB/s 
     |████████████████████████████████| 163 kB 69.2 MB/s 


In [ ]:
!pip install datasets --quiet
!pip install nltk --quiet

     |████████████████████████████████| 441 kB 4.2 MB/s 
     |████████████████████████████████| 115 kB 52.5 MB/s 
     |████████████████████████████████| 95 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 58.8 MB/s 
     |████████████████████████████████| 127 kB 56.3 MB/s 
     |████████████████████████████████| 115 kB 33.2 MB/s 


In [ ]:
!pip install -q rouge_score

In [ ]:
!pip install -q evaluate


     |████████████████████████████████| 72 kB 1.0 MB/s 


In [ ]:
!pip install -U spacy
!pip install -U spacy-lookups-data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98.5 MB 1.3 MB/s 


In [ ]:
!pip install -r rouge/requirements.txt
!pip install rouge-score
!pip install rouge_score

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'rouge/requirements.txt'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download en_core_web_lg

2022-11-06 16:00:47.097633: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 16 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


# 2.0 Import Libraries

In [ ]:
# NLTK
import re # relugar expression
import nltk # natural language toolkit for sentence tokenization and display
import string
import heapq
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.util import ngrams
import evaluate

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import os
import logging

import numpy as np
import tensorflow as tf
from tensorflow import keras

# Only log error messages
tf.get_logger().setLevel(logging.ERROR)

os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [ ]:
import evaluate

In [ ]:
import spacy
import pandas as pd

print(spacy.__version__)
print(pd.__version__)

3.4.2
1.3.5


In [ ]:
import pickle
import subprocess
import sys
import nltk
from nltk import Nonterminal, nonterminals, Production, CFG, PCFG

In [ ]:
#shift reduce parser example
from nltk.grammar import Nonterminal
from nltk.parse.api import ParserI
from nltk.tree import Tree

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
import time

In [ ]:
# Mount drive for saving model checkpoints, loading Task 2 data below

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 3.0 Load the dataset



In [ ]:
from datasets import load_dataset, load_metric


In [ ]:
# Load 1000 examples of CNN Dailymail from a CSV File

filepath = 'drive/My Drive/Colab_Notebooks_1/cnn_dailymail_1000.csv'
dataset = load_dataset('csv', data_files = filepath, split='train' )
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d8ef26a734f1246a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


Dataset({
    features: ['document', 'summary'],
    num_rows: 1000
})

# 4.0 Helper Functions

In [ ]:
def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.replace('\n', ' ')
    sentence = sentence.replace('?</s>', '?.')
    sentence = sentence.replace('</s>', '')
    sentence = sentence.replace("\'", "'")
    sentence = sentence.replace('--', '')
    sentence = sentence.replace('|', '')
    sentence = sentence.replace('/', '')
    sentence = sentence.replace('Dr.', 'Dr')
    sentence = sentence.replace('?.', '?. ')
    #sentence = sentence.replace('.', '. ')
    sentence = sentence.replace('!', '!. ')
    #sentence = sentence.replace('"', '')
    sentence = sentence.replace("This material may not be published, broadcast, rewritten, or redistributed.",'')
    #sentence = sentence.replace(' "', '')
    #sentence = sentence.replace('" ', '')
    return sentence

In [ ]:
def calculate_rouge(reference, predictions):
  rouge = evaluate.load('rouge')
  predictions = extracted_sentence_list
  references = orig_highlights_list
  results = rouge.compute(predictions=predictions,
                        references=references)
  print(results)

In [ ]:
"""
LexRank implementation
Source: https://github.com/crabcamp/lexrank/tree/dev
"""

import numpy as np
from scipy.sparse.csgraph import connected_components
from scipy.special import softmax
import logging

logger = logging.getLogger(__name__)

def degree_centrality_scores(
    similarity_matrix,
    threshold=None,
    increase_power=True,
):
    if not (
        threshold is None
        or isinstance(threshold, float)
        and 0 <= threshold < 1
    ):
        raise ValueError(
            '\'threshold\' should be a floating-point number '
            'from the interval [0, 1) or None',
        )

    if threshold is None:
        markov_matrix = create_markov_matrix(similarity_matrix)

    else:
        markov_matrix = create_markov_matrix_discrete(
            similarity_matrix,
            threshold,
        )

    scores = stationary_distribution(
        markov_matrix,
        increase_power=increase_power,
        normalized=False,
    )

    return scores


def _power_method(transition_matrix, increase_power=True, max_iter=10000):
    eigenvector = np.ones(len(transition_matrix))

    if len(eigenvector) == 1:
        return eigenvector

    transition = transition_matrix.transpose()

    for _ in range(max_iter):
        eigenvector_next = np.dot(transition, eigenvector)

        if np.allclose(eigenvector_next, eigenvector):
            return eigenvector_next

        eigenvector = eigenvector_next

        if increase_power:
            transition = np.dot(transition, transition)

    logger.warning("Maximum number of iterations for power method exceeded without convergence!")
    return eigenvector_next


def connected_nodes(matrix):
    _, labels = connected_components(matrix)

    groups = []

    for tag in np.unique(labels):
        group = np.where(labels == tag)[0]
        groups.append(group)

    return groups


def create_markov_matrix(weights_matrix):
    n_1, n_2 = weights_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'weights_matrix\' should be square')

    row_sum = weights_matrix.sum(axis=1, keepdims=True)

    # normalize probability distribution differently if we have negative transition values
    if np.min(weights_matrix) <= 0:
        return softmax(weights_matrix, axis=1)

    return weights_matrix / row_sum


def create_markov_matrix_discrete(weights_matrix, threshold):
    discrete_weights_matrix = np.zeros(weights_matrix.shape)
    ixs = np.where(weights_matrix >= threshold)
    discrete_weights_matrix[ixs] = 1

    return create_markov_matrix(discrete_weights_matrix)


def stationary_distribution(
    transition_matrix,
    increase_power=True,
    normalized=True,
):
    n_1, n_2 = transition_matrix.shape
    if n_1 != n_2:
        raise ValueError('\'transition_matrix\' should be square')

    distribution = np.zeros(n_1)

    grouped_indices = connected_nodes(transition_matrix)

    for group in grouped_indices:
        t_matrix = transition_matrix[np.ix_(group, group)]
        eigenvector = _power_method(t_matrix, increase_power=increase_power)
        distribution[group] = eigenvector

    if normalized:
        distribution /= n_1

    return distribution

In [ ]:
# Trigram Blocking for Extractive Summarization

def trigram_blocking(input_text):

  trigram_master_list=[]
  trigram_temp_list=[]
  clean_list = []
  trigram_flag = []
  output_list = input_text

  #output_list = ['oh my god', 'oh my god' ,'lovely day today', 'This year is good', 'Oh my god' ]

  for idx, sentence in enumerate(output_list):
    if idx == 0:
      token = nltk.word_tokenize(sentence.lower())  #tokenize your text and make it lowercase in onestep
      trigrams=ngrams(token,3)          # find all the trigram in text1
      #print(idx, sentence)
      for item in trigrams:
        #print(item)
        if item not in trigram_master_list:
          trigram_master_list.append(item)
      clean_list.append(sentence)
    elif idx > 0:
      token = nltk.word_tokenize(sentence.lower())  #tokenize your text and make it lowercase in onestep
      trigrams=ngrams(token,3)
      #print(idx, sentence) 
      for item in trigrams:
        #print(item)
        trigram_temp_list.append(item)
      trigram_flag = []  
      for trigram_temp in trigram_temp_list:
        if trigram_temp in trigram_master_list:
          trigram_flag.append("Y")
        else:
          trigram_flag.append("N")
      if "N" in trigram_flag:
        clean_list.append(sentence)
        for i in trigram_temp_list:
          trigram_master_list.append(i)

  #print(trigram_master_list)
  #print(clean_list)

  return (clean_list)

#5.0 Create a Dataset for Model Building and Evaluation

In [ ]:
# Generate a dataset of "x" examples for model evaulation
size_of_dataset = 1000 # Do Not Change the Value Here
small_dataset = dataset[0:size_of_dataset]


#6.0 Model Use Pre-Trained SBERT Sentence Transformer for Extractive Summarization 
    (NO FINE TUNING)

In [ ]:
# Install and Import SBertSummarizer
!pip install -U sentence-transformers
from summarizer.sbert import SBertSummarizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.7 MB/s 
     |████████████████████████████████| 1.3 MB 28.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=cb297f1683da80484fdd7f282e38bb15136ae75a36414fbe9354e1bf0f0e44f0
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
# load the pre-trained weights
model = SBertSummarizer('all-MiniLM-L6-v2')
# see model config
model

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
# Load metric
metric = load_metric("rouge")
metric

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_aggregator: Return aggregates if this is set to True
Retu

#7.0 Evaluate Rouge Across Eval Dataset

In [ ]:
# RougueL Scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge = evaluate.load('rouge')

In [ ]:
# Iterate through the dataset, extract the summary and compute RougeL, Cosine Similarity Scores

start = time.time()

# get article from dataset
input_article = small_dataset['document'][0:size_of_dataset]
#len(input_article)

# get summary from dataset
input_highlights = small_dataset['summary'][0:size_of_dataset]
#len(input_highlights)

# zip article and summary
zipped_input = zip(input_article, input_highlights)

# Empty List to Store Scores
rougeL_precision = []
rougeL_recall = []
rougeL_fmeasure = []

rouge_1_list = []
rouge_2_list = []
rouge_L_list = []

cosine_similarity_results = []

# Counter for Tracking Results
count = 1

# Extracted Highlights
output_highlights_list = []

# iterate Through the Eval Dataset
for input_article, input_highlights in zipped_input:
  
  print('Example:', count)

  # Tokenize the string texts
  
  source_article_list = nltk.sent_tokenize(input_article) # list of sentences
  source_highlight_list = nltk.sent_tokenize(input_highlights) # a list of sentences

  #print("source list")
  #print(source_article_list)
  #print(source_highlight_list)
  #print("")
  
# Join the list into a string
  formatted_source_article = " ".join(source_article_list) # a single string of sentences
  formatted_source_highlight = " ".join(source_highlight_list) # a string string of sentences

  #print("source formatted")
  #print(formatted_source_article)
  #print(formatted_source_highlight)
  #print("")

# generate extractive summary using the pretrainned model
  num_sentences = int(len(source_article_list) * 0.30) # 30% compression ration
  result = model(formatted_source_article, num_sentences=num_sentences)  # a single string of sentences
  
  #print("model summarization")
  #print(type(result))
  #print(result)
  #print("")

# note result is a string returned by model
  summary = "".join(result)
  
  #print("string join summary:")
  #print(summary)
  #print("")

# Take the string from summary and convert to list of strings for each sentence
  extracted_sentence_list = nltk.sent_tokenize(summary) # convert a single string to sentences
  
  #print("extracted sentence list")
  #print(extracted_sentence_list)
  #print("")

# Apply Trigram Blocking
  result_list = trigram_blocking(extracted_sentence_list)

  #print("trigram result list:")
  #print(result_list)
  #print("")
  
  summary = " ".join(result_list)
  
  #print("trigram summary:")
  #print(summary)
  #print("")
  
  output_highlights_list.append(summary)

# Define predictions and references for score calculation
  predictions = " ".join(result_list) # single string of sentences
  references = formatted_source_highlight # single string of sentences
  
  #print("predictions:")
  #print(predictions)
  #print("")
  #print("references")
  #print(references)
  #print("")



# Calculate Scores
  rougeL_scores = scorer.score(predictions,
                      references)

  pred = [predictions]
  ref = [references]

  rouge_results = rouge.compute(predictions=pred, references=ref)
  print(rouge_results)

  rouge_1_score = (rouge_results['rouge1'])
  rouge_2_score = (rouge_results['rouge2'])
  rouge_L_score = (rouge_results['rougeL'])

  rouge_1_list.append(rouge_1_score)
  rouge_2_list.append(rouge_2_score)
  rouge_L_list.append(rouge_L_score)

  precision = (rougeL_scores['rougeL'].precision)
  recall = (rougeL_scores['rougeL'].recall)
  fmeasure = (rougeL_scores['rougeL'].fmeasure)

  rougeL_precision.append(precision)
  rougeL_recall.append(recall)
  rougeL_fmeasure.append(fmeasure)

  # calculate cosine similarity
  doc1 = nlp((" ".join(predictions)))
  doc2 = nlp((" ".join(references)))
  cosine_similarity = doc1.similarity(doc2)
  cosine_similarity_results.append(cosine_similarity)

  count = count + 1


Example: 1
{'rouge1': 0.28729281767955805, 'rouge2': 0.2122905027932961, 'rougeL': 0.26519337016574585, 'rougeLsum': 0.26519337016574585}
Example: 2
{'rouge1': 0.24060150375939846, 'rouge2': 0.0909090909090909, 'rougeL': 0.12781954887218044, 'rougeLsum': 0.1879699248120301}
Example: 3
{'rouge1': 0.13999999999999999, 'rouge2': 0.040268456375838924, 'rougeL': 0.08666666666666666, 'rougeLsum': 0.10666666666666667}
Example: 4
{'rouge1': 0.2318840579710145, 'rouge2': 0.07352941176470588, 'rougeL': 0.14492753623188404, 'rougeLsum': 0.14492753623188404}
Example: 5
{'rouge1': 0.1563517915309446, 'rouge2': 0.059016393442622946, 'rougeL': 0.09771986970684039, 'rougeLsum': 0.09771986970684039}
Example: 6
{'rouge1': 0.17518248175182483, 'rouge2': 0.03676470588235294, 'rougeL': 0.10218978102189781, 'rougeLsum': 0.1094890510948905}
Example: 7
{'rouge1': 0.12923076923076923, 'rouge2': 0.01238390092879257, 'rougeL': 0.07384615384615385, 'rougeLsum': 0.07384615384615385}
Example: 8
{'rouge1': 0.3857868

In [ ]:
# Print Compute Time
print('\nTime:', time.time() - start)


Time: 1103.1753194332123


In [ ]:
# Clean up the extracted list
output_highlights_list = list(((map(preprocess, output_highlights_list))))

In [ ]:
# Export extractive summary to a CSV

# get article from dataset
input_article_list = small_dataset['document'][0:size_of_dataset]

# get summary from dataset
input_highlights_list = small_dataset['summary'][0:size_of_dataset]

df = pd.DataFrame(list(zip(input_article_list, input_highlights_list, output_highlights_list)),
                  columns = ['orig_article', 'orig_summary', 'extracted_summary'])

# Edit this filepath to wherever you saved the data in your Drive
filepath = 'drive/My Drive/Colab_Notebooks_1/model_1a_extracted_CNNDaily1000.csv'

df.to_csv(filepath,index = False)

In [ ]:
# read back the csv file
data_import = pd.read_csv(filepath)        
#data_import.rename(columns = {'0':'orig_article', '1':'orig_summary', '2':'extracted_summary'}, inplace = True)

col1 = data_import.orig_article.values.tolist()
col2 = data_import.orig_summary.values.tolist()
col3 = data_import.extracted_summary.values.tolist()
   


In [ ]:
# Validate data_import
print(col1[0])
print("")

print(col2[0])
print("")

print(col3[0])


LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how

In [ ]:
# Calculate Mean Rouge for Dataset

print("RougeL Precision Scores")
print(rougeL_precision)
print(len(rougeL_precision))
print(np.mean(np.asarray(rougeL_precision)))
print("")

print("RougeL Recall Scores")
print(rougeL_recall)
print(len(rougeL_precision))
print(np.mean(np.asarray(rougeL_recall)))
print("")

print("RougeL Fmeasure Scores")
print(rougeL_fmeasure)
print(len(rougeL_fmeasure))
print(np.mean(np.asarray(rougeL_fmeasure)))
print("")

print("Rouge 1 Scores")
print(rouge_1_list)
print(len(rouge_1_list))
print(np.mean(np.asarray(rouge_1_list)))
print("")

print("Rouge 2 Scores")
print(rouge_2_list)
print(len(rouge_2_list))
print(np.mean(np.asarray(rouge_2_list)))
print("")

print("Rouge L Scores")
print(rouge_L_list)
print(len(rouge_L_list))
print(np.mean(np.asarray(rouge_L_list)))
print("")







RougeL Precision Scores
[0.6153846153846154, 0.3469387755102041, 0.3170731707317073, 0.4583333333333333, 0.4146341463414634, 0.32558139534883723, 0.2765957446808511, 0.5238095238095238, 0.48484848484848486, 0.42857142857142855, 0.6486486486486487, 0.3888888888888889, 0.3888888888888889, 0.2903225806451613, 0.1956521739130435, 0.3877551020408163, 0.34615384615384615, 0.14285714285714285, 0.3111111111111111, 0.6285714285714286, 0.4883720930232558, 0.48148148148148145, 0.2608695652173913, 0.4864864864864865, 0.41818181818181815, 0.34375, 0.6666666666666666, 0.125, 0.4, 0.23684210526315788, 0.6222222222222222, 0.574468085106383, 0.3225806451612903, 0.40540540540540543, 0.41509433962264153, 0.32432432432432434, 0.4888888888888889, 0.3333333333333333, 0.6046511627906976, 0.3191489361702128, 0.3409090909090909, 0.3225806451612903, 0.4117647058823529, 0.32653061224489793, 0.4864864864864865, 0.5813953488372093, 0.3125, 0.3170731707317073, 0.2857142857142857, 0.45454545454545453, 0.333333333333

In [ ]:
# Calculate Mean Cosine Similarity

print("Cosine Similarity")
print(len(cosine_similarity_results))
print(np.mean(np.asarray(cosine_similarity_results)))
print("")


Cosine Similarity
1000
0.9928218872330346



In [ ]:
dataset = load_dataset('csv', data_files = filepath, split='train' )

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-9e104c0f19bc94bc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


In [ ]:
dataset

Dataset({
    features: ['orig_article', 'orig_summary', 'extracted_summary'],
    num_rows: 1000
})